In [1]:
import scipy.optimize
from pandas import *
from numpy import *

In [2]:
##################################################################
# First, some helper files from 
#https://github.com/omartinsky/QuantAndFinancial/blob/master/black_litterman/black_litterman.ipynb
##################################################################

# Calculates portfolio mean return
def port_mean(W, R):
    return sum(R * W)

# Calculates portfolio variance of returns
def port_var(W, C):
    return dot(dot(W, C), W)

# Combination of the two functions above - mean and variance of returns calculation
def port_mean_var(W, R, C):
    return port_mean(W, R), port_var(W, C)

In [3]:
##################################################################
#This comes from https://www.quantandfinancial.com/2013/07/mean-variance-portfolio-optimization.html 
##################################################################

# define the utility function
def fitness(W, R, C, rf, delta):
    # calculate mean/variance of the portfolio
    mean, var = port_mean_var(W, R, C)  

    # utility function = Sharpe ratio
    #util = (mean - rf) / sqrt(var)      
    #return 1/util                   # invert bc the function below minimizes, not maximizes
                                     # or just multiply by negative 1

    # mean-var utility
    util = mean - (delta/2) * var
    return -util                     # max(utility) = minimize(-utility)

# Given risk-free rate, assets returns and covariances, this function calculates
# weights of tangency portfolio with respect to sharpe ratio maximization
def solve_weights(R, C, rf, delta):
        
    # a lil more set up
    n = len(R)                          # number of risky assets
    W = np.ones([n])/n                     # start with equal weights
    
    # constraints on the portfolio weights (replace with None to remove)
    b_ = [(0.,1.) for i in range(n)]    # weights between 0%..100%. 
                                        # No leverage, no shorting     

    # overall portfolio constraint 
    c_ = ({'type':'eq', 'fun': 
           lambda W: sum(W)-1. })       # Sum of weights = 100%
    
    optimized = scipy.optimize.minimize(fitness, W, (R, C, rf, delta), 
                method='SLSQP', constraints=c_, bounds=b_)  
    
    if not optimized.success: 
        raise BaseException(optimized.message)
        
    return optimized.x  # Return optimized weights

In [4]:
##################################################################
# Example (you'll need to estimate R and C in practice, here fake)
##################################################################

R = [.08,.15]                          # bond and equity
C = [[.05,.7],[.7,.40]]                # covariance matrix
rf = .01                               # only matters with sharpe utility, but still have to pick a # for mean-var
delta = .1                             # risk aversion parameter
x= solve_weights(R, C, rf, delta)
print("weight",x)
print("mu,var",port_mean_var(x, R, C))
print("util  ",fitness(x, R, C, rf, delta))

print('\n\n','='*40,'\n Low risk aversion --> all equity portfolio\n','='*40)
delta = .1                             # risk aversion parameter
x= solve_weights(R, C, rf, delta)
print(" weights: ",x)

print('\n\n','='*40,'\n High risk aversion --> all debt portfolio\n','='*40)
delta = 100                             # risk aversion parameter
x= solve_weights(R, C, rf, delta)
print(" weights: ",x)


print('\n\n','='*40,'\n In the middle --> mixed portfolio\n','='*40)
delta = .4                             # risk aversion parameter
x= solve_weights(R, C, rf, delta)
print(" weights: ",x)


weight [0. 1.]
mu,var (0.15, 0.4)
util   -0.13


 Low risk aversion --> all equity portfolio
 weights:  [0. 1.]


 High risk aversion --> all debt portfolio
 weights:  [1. 0.]


 In the middle --> mixed portfolio
 weights:  [0.5 0.5]
